In [90]:
from agents.llm_client import get_llm_client

In [85]:
bumrah_balls['wicket_type'].value_counts()

wicket_type
caught               116
bowled                46
lbw                   17
run out               16
caught and bowled      4
Name: count, dtype: int64

In [83]:
dismissal_counts

,wicket_type,wickets
0,bowled,46
1,caught,116
2,lbw,17


In [72]:
final_result

[{'phase': 'middle', 'count': 95},
 {'phase': 'powerplay', 'count': 49},
 {'phase': 'death', 'count': 46}]

In [59]:
innings_above_30['fifty'].value_counts()

fifty
True     71
False    52
Name: count, dtype: int64

In [86]:
df = pd.read_parquet("data/venues.parquet")

In [87]:
df

,venue_id,venue_name,city
0,venue_78422,Arun Jaitley Stadium,Delhi
1,venue_24296,"Arun Jaitley Stadium, Delhi",Delhi
2,venue_60645,Barabati Stadium,Cuttack
3,venue_80458,"Barsapara Cricket Stadium, Guwahati",Guwahati
4,venue_63956,Bharat Ratna Shri Atal Bihari Vajpayee Ekana C...,Lucknow
...,...,...,...
58,venue_71801,SuperSport Park,Centurion
59,venue_18978,"Vidarbha Cricket Association Stadium, Jamtha",Nagpur
60,venue_62745,Wankhede Stadium,Mumbai
61,venue_60873,"Wankhede Stadium, Mumbai",Mumbai


In [26]:
final_result

{'player': 'JJ Bumrah', 'wickets': 203}

In [132]:
team_avg_runs['run_rate'] = team_avg_runs['average_runs']*6
team_avg_runs['avg_total'] = team_avg_runs['run_rate']*5

In [134]:
team_avg_runs

,batting_team,average_runs,run_rate,avg_total
0,Delhi Capitals,1.711268,10.267606,51.338028
1,Gujarat Giants,1.508380,9.050279,45.251397
2,Mumbai Indians,1.840659,11.043956,55.219780
3,Royal Challengers Bengaluru,1.769912,10.619469,53.097345
4,UP Warriorz,1.423313,8.539877,42.699387


In [83]:
nb_runs = 0
nb_balls = 0
for season in final_result:
    nb_runs += season['non_boundary_runs']
    nb_balls += season['non_boundary_balls']
print(nb_runs)
print(nb_balls)

115
351


In [5]:
import sqlite3
import pandas as pd
import sys
import os

def load_db_to_df(db_path, query=None, table=None):
    """
    Load data from a SQLite database into a pandas DataFrame.
    
    Args:
        db_path (str): Path to the SQLite database file.
        query (str, optional): SQL query to execute.
        table (str, optional): Table name to load entire content from.
        
    Returns:
        pd.DataFrame: DataFrame containing the result, or None if error.
    """
    if not os.path.exists(db_path):
        print(f"Error: Database file not found at {db_path}")
        return None

    try:
        # Create a connection to the database
        conn = sqlite3.connect(db_path)
        
        if query:
            print(f"Executing query: {query}")
            df = pd.read_sql_query(query, conn)
        elif table:
            print(f"Loading table: {table}")
            df = pd.read_sql_query(f"SELECT * FROM {table}", conn)
        else:
            # If no query or table specified, list tables
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
            tables = cursor.fetchall()
            
            print(f"Connected to {db_path}")
            print("Tables found in database:")
            for t in tables:
                print(f" - {t[0]}")
            
            if not tables:
                print("No tables found.")
                conn.close()
                return None
                
            # Default to loading the first table if available
            first_table = tables[0][0]
            print(f"\nNo table or query specified. Loading first table: '{first_table}'")
            df = pd.read_sql_query(f"SELECT * FROM {first_table}", conn)

        conn.close()
        return df

    except Exception as e:
        print(f"Error reading database: {e}")
        if 'conn' in locals():
            conn.close()
        return None


In [7]:
df = load_db_to_df("data/chatbot_history.db", table="interactions")

Loading table: interactions


In [8]:
df

,id,timestamp,query,success,expander_output,decomposer_output,generator_output,formatter_output,final_result_json,error_message,response_summary
0,1,2026-01-30T00:48:24.348213,Test query,1,"{""model"": ""test-model"", ""usage"": {""prompt_toke...","{""model"": ""test-model"", ""usage"": {""prompt_toke...","{""model"": ""test-model"", ""usage"": {""prompt_toke...","{""model"": ""test-model"", ""usage"": {""prompt_toke...","{""success"": true, ""answer"": {""summary"": ""Summa...",None,Summary text
1,2,2026-01-30T00:49:18.095684,Wickets taken by Jasprit bumrah,1,"{""model"": ""gemma-3-27b-it"", ""usage"": {""complet...","{""model"": ""gemma-3-27b-it"", ""usage"": {""complet...","{""model"": ""gemma-3-27b-it"", ""usage"": {""complet...","{""model"": ""gemma-3-27b-it"", ""usage"": {""complet...","{""query"": ""Wickets taken by Jasprit bumrah"", ""...",None,"Jasprit Bumrah, the spearhead of the Indian at..."
